In [1]:
import warnings

warnings.filterwarnings("ignore")

import pyspark.sql
from pyspark.sql.functions import *

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from wrangle import wrangle_311

spark = pyspark.sql.SparkSession.builder.getOrCreate()

Answer the questions below by using a combination of the techniques discussed in the lesson that you think is appropriate.

In [2]:
df = wrangle_311(spark)

[wrangle.py] reading case.csv
[wrangle.py] handling data types
[wrangle.py] parsing dates
[wrangle.py] adding features
[wrangle.py] joining departments


In [6]:
df.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('num_weeks_late', 'double'),
 ('zipcode', 'string'),
 ('case_age', 'int'),
 ('days_to_closed', 'int'),
 ('case_lifetime', 'int'),
 ('department', 'string'),
 ('dept_subject_to_SLA', 'boolean')]

How many different cases are there, by department?

In [9]:
df.groupBy('department').count().sort('count').show()

+--------------------+------+
|          department| count|
+--------------------+------+
|        City Council|    33|
|    Customer Service|  2849|
|        Metro Health|  5163|
|  Parks & Recreation| 19907|
|Trans & Cap Impro...| 96193|
|Animal Care Services|116915|
|         Solid Waste|279270|
|DSD/Code Enforcement|321374|
+--------------------+------+



Does the percentage of cases that are late vary by department?

In [15]:
dept_late = df.crosstab("department", "case_late")

In [19]:
dept_late.dtypes

[('department_case_late', 'string'), ('false', 'bigint'), ('true', 'bigint')]

In [21]:
dept_late.select(
    dept_late.department_case_late, 
    (dept_late.true/(dept_late.true + dept_late.false)).alias("Percentage of Cases Late")
).show()

+--------------------+------------------------+
|department_case_late|Percentage of Cases Late|
+--------------------+------------------------+
|        City Council|                     0.0|
|         Solid Waste|      0.1179682744297633|
|Trans & Cap Impro...|     0.05625149439148379|
|  Parks & Recreation|     0.19073692670919776|
|    Customer Service|      0.7055107055107055|
|DSD/Code Enforcement|      0.0816338596152769|
|Animal Care Services|     0.19908480520035923|
|        Metro Health|     0.16056556265736974|
+--------------------+------------------------+



On average, how late are the late cases by department?

In [23]:
df.filter(df.num_days_late>0).groupBy('department').mean('num_days_late').show()

+--------------------+------------------+
|          department|avg(num_days_late)|
+--------------------+------------------+
|         Solid Waste| 7.186821906120906|
|Animal Care Services|23.458633245820135|
|Trans & Cap Impro...| 10.60306468031694|
|  Parks & Recreation|22.348910457867508|
|    Customer Service| 87.68385942150395|
|        Metro Health| 6.543813315547647|
|DSD/Code Enforcement|49.384287053588935|
+--------------------+------------------+



What is the service type that is the most late? 

In [32]:
df.filter(df.num_days_late>0).groupBy(
    'service_request_type').mean(
    'num_days_late').sort(desc(
    "avg(num_days_late)")).first().service_request_type

'Zoning: Recycle Yard'

Just for Parks & Rec?

In [33]:
df.filter(df.num_days_late>0).where(df['department'] == 'Parks & Recreation').groupBy(
    'service_request_type').mean(
    'num_days_late').sort(desc(
    "avg(num_days_late)")).first().service_request_type

'Amenity Park Improvement'

For the DSD/Code Enforcement department, what are the most common service request types? Look at other departments too.

In [39]:
df.where(df['department'] == "DSD/Code Enforcement").groupBy('service_request_type').count().sort(desc('count')).head(5)

[Row(service_request_type='Overgrown Yard/Trash', count=65895),
 Row(service_request_type='Bandit Signs', count=32910),
 Row(service_request_type='Front Or Side Yard Parking', count=28794),
 Row(service_request_type='Junk Vehicle On Private Property', count=21473),
 Row(service_request_type='Alley-Way Maintenance', count=20214)]

Does whether or not its a weekend matter for when a case is opened/closed?

On average, how many cases are opened a day for the Customer Service department?

Does the number of service requests for the solid waste department vary by day of the week?